In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import country_converter as coco
from selenium import webdriver as driver
from geopy.geocoders import Nominatim
import folium

In [2]:
#scraping data from a website
df = pd.read_html('https://f1.fandom.com/wiki/Circuits')[0]
df = df[df["Season(s)"].str.contains("present") == True]
df = df[df["Location"].str.contains("Miami Gardens, FL")==False]
df = df[df["Location"].str.contains("Zandvoort")==False]
tracks = df.drop(columns=["Type", "Race(s)", "Season(s)", "Races held"])
tracks

,Circuit,Location,Country
4,Albert Park,Melbourne,AUS
7,Bahrain International Circuit,Sakhir,BHR
9,Baku City Circuit,Baku,AZE
10,Circuit de Barcelona-Catalunya,Montmeló,ESP
19,Circuit of the Americas,"Austin, TX",USA
26,Autodromo Enzo e Dino Ferrari,Imola,ITA
30,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN
33,Autódromo Hermanos Rodríguez,Mexico City,MEX
35,Hungaroring,Mogyoród,HUN
46,Autódromo José Carlos Pace,São Paulo,BRA


In [3]:
#generating the country names from their codes using country_converter
tracks['Country Name'] = tracks.Country.apply(lambda x: coco.convert(names=x, to='name_short', not_found=None))
tracks

SIN not found in ISO3
MON not found in ISO3
UAE not found in ISO3


,Circuit,Location,Country,Country Name
4,Albert Park,Melbourne,AUS,Australia
7,Bahrain International Circuit,Sakhir,BHR,Bahrain
9,Baku City Circuit,Baku,AZE,Azerbaijan
10,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain
19,Circuit of the Americas,"Austin, TX",USA,United States
26,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy
30,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada
33,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico
35,Hungaroring,Mogyoród,HUN,Hungary
46,Autódromo José Carlos Pace,São Paulo,BRA,Brazil


In [4]:
#as the original codes were not in ISO3 format, some codes needed to be converted manually
df = tracks.replace({'Country Name':{'SIN': 'Singapore','MON': 'Monaco', 'UAE': 'United Arab Emirates', 'NED': 'Netherlands'}})
df

,Circuit,Location,Country,Country Name
4,Albert Park,Melbourne,AUS,Australia
7,Bahrain International Circuit,Sakhir,BHR,Bahrain
9,Baku City Circuit,Baku,AZE,Azerbaijan
10,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain
19,Circuit of the Americas,"Austin, TX",USA,United States
26,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy
30,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada
33,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico
35,Hungaroring,Mogyoród,HUN,Hungary
46,Autódromo José Carlos Pace,São Paulo,BRA,Brazil


In [5]:
#getting attendance data
data = pd.read_html('https://f1destinations.com/2017-f1-attendance-figures/')[0]
data = data[data["Race"].str.contains("TOTAL")==False]
data = data.drop(columns=["% Change"])

columns_titles = ["Race","2016 Attendance","2017 Attendance"]

data = data.reindex(columns=columns_titles)
data["Growth (%)"] = (((data["2017 Attendance"] / data["2016 Attendance"]) - 1) * 100).astype(int)
data['Race'] = data['Race'].str.replace('\d+', '', regex=True).str.replace('.', '', regex=True).str.replace(' ', '', regex=True)
data = data.replace({'Race':{'USA': 'United States', 'AbuDhabi': 'United Arab Emirates', 'GreatBritain': 'United Kingdom'}})
data

,Race,2016 Attendance,2017 Attendance,Growth (%)
0,Canada,300000,360000,19
1,United Kingdom,350000,344500,-1
2,Mexico,339967,337043,0
3,Australia,271800,296600,9
4,Belgium,233730,265000,13
5,Singapore,219000,260000,18
6,United States,269889,258000,-4
7,Monaco,200000,200000,0
8,Hungary,176000,199000,13
9,United Arab Emirates,195000,195000,0


In [6]:
join = pd.merge(df, data, left_on='Country Name', right_on='Race', how='left')
join

,Circuit,Location,Country,Country Name,Race,2016 Attendance,2017 Attendance,Growth (%)
0,Albert Park,Melbourne,AUS,Australia,Australia,271800,296600,9
1,Bahrain International Circuit,Sakhir,BHR,Bahrain,Bahrain,92000,93000,1
2,Baku City Circuit,Baku,AZE,Azerbaijan,Azerbaijan,30000,71541,138
3,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain,Spain,165029,177984,7
4,Circuit of the Americas,"Austin, TX",USA,United States,United States,269889,258000,-4
5,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy,Italy,147500,185000,25
6,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada,Canada,300000,360000,19
7,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico,Mexico,339967,337043,0
8,Hungaroring,Mogyoród,HUN,Hungary,Hungary,176000,199000,13
9,Autódromo José Carlos Pace,São Paulo,BRA,Brazil,Brazil,136410,141218,3


In [7]:
geolocator = Nominatim(user_agent='test')
join['gcode'] = join['Location'].apply(geolocator.geocode)
join['lat'] = [g.latitude for g in join.gcode]
join['long'] = [g.longitude for g in join.gcode]
final = join.drop(columns=["gcode"])
final

,Circuit,Location,Country,Country Name,Race,2016 Attendance,2017 Attendance,Growth (%),lat,long
0,Albert Park,Melbourne,AUS,Australia,Australia,271800,296600,9,-37.814218,144.963161
1,Bahrain International Circuit,Sakhir,BHR,Bahrain,Bahrain,92000,93000,1,32.953941,65.540016
2,Baku City Circuit,Baku,AZE,Azerbaijan,Azerbaijan,30000,71541,138,40.375443,49.832675
3,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain,Spain,165029,177984,7,41.551519,2.248081
4,Circuit of the Americas,"Austin, TX",USA,United States,United States,269889,258000,-4,30.271129,-97.743700
5,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy,Italy,147500,185000,25,44.353515,11.714123
6,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada,Canada,300000,360000,19,45.503182,-73.569806
7,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico,Mexico,339967,337043,0,19.432630,-99.133178
8,Hungaroring,Mogyoród,HUN,Hungary,Hungary,176000,199000,13,47.603742,19.247409
9,Autódromo José Carlos Pace,São Paulo,BRA,Brazil,Brazil,136410,141218,3,-23.550651,-46.633382


In [18]:
#building the map
def find_color(growth):
    if (growth > 20):
        return 'green'
    if (growth > 10):
        return 'yellow'
    else:
        return 'red'

map_clusters = folium.Map(zoom_start=4)

for lat, long, att, growth in zip(final['lat'], final['long'], final['2017 Attendance'], final['Growth (%)']):
    folium.CircleMarker(
    [lat, long],
    color='black',
    fill_color=find_color(growth),
    weight=1,
    fill_opacity = 0.6,
    radius = att/20000).add_to(map_clusters)
    
map_clusters

In [9]:
from tkinter import *
from tkinter import filedialog
from tkinter import Tk
import json

# Quickly get rid of the root window popup
root = Tk()
root.withdraw()

# Use Filedialog.askopenfilename to open a dialog window where you can select your credentials file.
filepath = filedialog.askopenfilename()
file = open(filepath, 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

api_key = credentials['api_key']
client_id = credentials['client_id']
client_secret = credentials['client_secret']
test = credentials['test']
category_id = '17006'
version = 1

print("test:", test)

test: test


In [10]:
concat = final[['Location', 'lat', 'long']].copy()
concat["ll"] = concat["lat"].astype(str) + ',' + concat["long"].astype(str)
cc = concat.drop(columns=['lat', 'long'])
cc

,Location,ll
0,Melbourne,"-37.8142176,144.9631608"
1,Sakhir,"32.953941,65.540016"
2,Baku,"40.3754434,49.8326748"
3,Montmeló,"41.5515189,2.2480812"
4,"Austin, TX","30.2711286,-97.7436995"
5,Imola,"44.3535145,11.7141233"
6,"Montréal, Quebec","45.5031824,-73.5698065"
7,Mexico City,"19.4326296,-99.1331785"
8,Mogyoród,"47.6037418,19.24740913013881"
9,São Paulo,"-23.5506507,-46.6333824"


In [11]:
headers = {
    "Accept": "application/json",
    "Authorization": api_key
}
#get the number of dealerships in a 5km radius for each track using Foursquare API
coordinates = cc['ll'].tolist()
counts = []
for i in coordinates:
    url = "https://api.foursquare.com/v3/places/search?ll={}&radius=5000&categories=17006&limit=50".format(i)
    response = requests.get(url, headers=headers).json()['results']
    num = len(response)
    counts.append(num)
countsdf = pd.DataFrame(counts)
countsdf

,0
0,50
1,0
2,31
3,50
4,50
5,36
6,50
7,50
8,2
9,50


In [12]:
final['Number of Dealerships'] = countsdf[0]
final

,Circuit,Location,Country,Country Name,Race,2016 Attendance,2017 Attendance,Growth (%),lat,long,Number of Dealerships
0,Albert Park,Melbourne,AUS,Australia,Australia,271800,296600,9,-37.814218,144.963161,50
1,Bahrain International Circuit,Sakhir,BHR,Bahrain,Bahrain,92000,93000,1,32.953941,65.540016,0
2,Baku City Circuit,Baku,AZE,Azerbaijan,Azerbaijan,30000,71541,138,40.375443,49.832675,31
3,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain,Spain,165029,177984,7,41.551519,2.248081,50
4,Circuit of the Americas,"Austin, TX",USA,United States,United States,269889,258000,-4,30.271129,-97.743700,50
5,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy,Italy,147500,185000,25,44.353515,11.714123,36
6,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada,Canada,300000,360000,19,45.503182,-73.569806,50
7,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico,Mexico,339967,337043,0,19.432630,-99.133178,50
8,Hungaroring,Mogyoród,HUN,Hungary,Hungary,176000,199000,13,47.603742,19.247409,2
9,Autódromo José Carlos Pace,São Paulo,BRA,Brazil,Brazil,136410,141218,3,-23.550651,-46.633382,50


In [13]:
final.loc[final['Country Name'] == 'Bahrain', 'Number of Dealerships'] = 1
final['Ratio'] = (final["2017 Attendance"] / final['Number of Dealerships']).round()
final

,Circuit,Location,Country,Country Name,Race,2016 Attendance,2017 Attendance,Growth (%),lat,long,Number of Dealerships,Ratio
0,Albert Park,Melbourne,AUS,Australia,Australia,271800,296600,9,-37.814218,144.963161,50,5932.0
1,Bahrain International Circuit,Sakhir,BHR,Bahrain,Bahrain,92000,93000,1,32.953941,65.540016,1,93000.0
2,Baku City Circuit,Baku,AZE,Azerbaijan,Azerbaijan,30000,71541,138,40.375443,49.832675,31,2308.0
3,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain,Spain,165029,177984,7,41.551519,2.248081,50,3560.0
4,Circuit of the Americas,"Austin, TX",USA,United States,United States,269889,258000,-4,30.271129,-97.743700,50,5160.0
5,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy,Italy,147500,185000,25,44.353515,11.714123,36,5139.0
6,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada,Canada,300000,360000,19,45.503182,-73.569806,50,7200.0
7,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico,Mexico,339967,337043,0,19.432630,-99.133178,50,6741.0
8,Hungaroring,Mogyoród,HUN,Hungary,Hungary,176000,199000,13,47.603742,19.247409,2,99500.0
9,Autódromo José Carlos Pace,São Paulo,BRA,Brazil,Brazil,136410,141218,3,-23.550651,-46.633382,50,2824.0


In [14]:
pci = pd.read_csv('income.csv')
pci

,country,medianIncome,meanIncome,gdpPerCapitaPPP,pop2022
0,Luxembourg,26321,31376,124590,642.371
1,United Arab Emirates,24292,27017,70089,10081.785
2,Norway,22684,25272,70005,5511.370
3,Switzerland,21490,25787,72376,8773.637
4,United States,19306,25332,65297,334805.269
...,...,...,...,...,...
157,Burundi,475,640,784,12624.840
158,Madagascar,398,567,1719,29178.077
159,DR Congo,395,548,1146,95240.792
160,India,0,1314,6996,1406631.776


In [15]:
fin = pd.merge(final, pci, left_on='Country Name', right_on='country', how='left').drop(columns=['country', 'medianIncome', 'gdpPerCapitaPPP', 'pop2022'])
fin


,Circuit,Location,Country,Country Name,Race,2016 Attendance,2017 Attendance,Growth (%),lat,long,Number of Dealerships,Ratio,meanIncome
0,Albert Park,Melbourne,AUS,Australia,Australia,271800,296600,9,-37.814218,144.963161,50,5932.0,21329.0
1,Bahrain International Circuit,Sakhir,BHR,Bahrain,Bahrain,92000,93000,1,32.953941,65.540016,1,93000.0,NaN
2,Baku City Circuit,Baku,AZE,Azerbaijan,Azerbaijan,30000,71541,138,40.375443,49.832675,31,2308.0,3851.0
3,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain,Spain,165029,177984,7,41.551519,2.248081,50,3560.0,13822.0
4,Circuit of the Americas,"Austin, TX",USA,United States,United States,269889,258000,-4,30.271129,-97.743700,50,5160.0,25332.0
5,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy,Italy,147500,185000,25,44.353515,11.714123,36,5139.0,15547.0
6,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada,Canada,300000,360000,19,45.503182,-73.569806,50,7200.0,22042.0
7,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico,Mexico,339967,337043,0,19.432630,-99.133178,50,6741.0,4929.0
8,Hungaroring,Mogyoród,HUN,Hungary,Hungary,176000,199000,13,47.603742,19.247409,2,99500.0,8613.0
9,Autódromo José Carlos Pace,São Paulo,BRA,Brazil,Brazil,136410,141218,3,-23.550651,-46.633382,50,2824.0,7654.0


In [16]:
fin.loc[fin['Country Name'] == 'Bahrain', 'meanIncome'] = 47606
fin.loc[fin['Country Name'] == 'Singapore', 'meanIncome'] = 4680 * 12
fin.loc[fin['Country Name'] == 'Monaco', 'meanIncome'] = 161000
df = fin.drop(columns=['Race'])
df = df.rename(columns={"meanIncome": "Income"})
df

,Circuit,Location,Country,Country Name,2016 Attendance,2017 Attendance,Growth (%),lat,long,Number of Dealerships,Ratio,Income
0,Albert Park,Melbourne,AUS,Australia,271800,296600,9,-37.814218,144.963161,50,5932.0,21329.0
1,Bahrain International Circuit,Sakhir,BHR,Bahrain,92000,93000,1,32.953941,65.540016,1,93000.0,47606.0
2,Baku City Circuit,Baku,AZE,Azerbaijan,30000,71541,138,40.375443,49.832675,31,2308.0,3851.0
3,Circuit de Barcelona-Catalunya,Montmeló,ESP,Spain,165029,177984,7,41.551519,2.248081,50,3560.0,13822.0
4,Circuit of the Americas,"Austin, TX",USA,United States,269889,258000,-4,30.271129,-97.743700,50,5160.0,25332.0
5,Autodromo Enzo e Dino Ferrari,Imola,ITA,Italy,147500,185000,25,44.353515,11.714123,36,5139.0,15547.0
6,Circuit Gilles Villeneuve,"Montréal, Quebec",CAN,Canada,300000,360000,19,45.503182,-73.569806,50,7200.0,22042.0
7,Autódromo Hermanos Rodríguez,Mexico City,MEX,Mexico,339967,337043,0,19.432630,-99.133178,50,6741.0,4929.0
8,Hungaroring,Mogyoród,HUN,Hungary,176000,199000,13,47.603742,19.247409,2,99500.0,8613.0
9,Autódromo José Carlos Pace,São Paulo,BRA,Brazil,136410,141218,3,-23.550651,-46.633382,50,2824.0,7654.0


In [17]:
df.to_csv('data.csv', index=False)